In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [34]:
cd drive/My\ Drive/Fake\ Orders/

[Errno 2] No such file or directory: 'drive/My Drive/Fake Orders/'
/content/drive/My Drive/Fake Orders


In [35]:
ls

 20191206_spaces_info.csv        20191207_spaces_info.csv
'201912.06 space_time_fix.csv'  'Fake Orders empty time.csv'


## Read the Spaces inforomation

In [0]:
import csv
import datetime

##### Read the spaces information #####

with open('20191207_spaces_info.csv', newline='', encoding='big5') as space_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  spaces_info = csv.DictReader(space_file)
  
  # 以迴圈讀取指定欄位
  locate_id = []; space_id = []; price = []; size = []; zipcode = [];
  open_day = []; close_day = []; open_time = []; close_time = []; creat_time = []; mrt = [];
  for i in spaces_info:
    locate_id.append(i['locate_id'])
    space_id.append(i['space_id'])
    price.append(i['price'])
    size.append(i['size'])
    zipcode.append(i['zipcode'])
    open_day.append(i['open_day'])
    close_day.append(i['close_day'])
    open_time.append(i['open_time'])
    close_time.append(i['close_time'])
    creat_time.append(i['creat_time'])
    mrt.append(i['mrt'])
  space_file.close()

#space_id = list(map(int, space_id))
price = list(map(int, price))
size = list(map(int, size))
zipcode = list(map(int, zipcode))
open_day = list(map(int, open_day))
close_day = list(map(int, close_day))

## Generate Fake Orders

In [0]:
format = '%H:%M'
##### Generate the Fake Orders #####
with open('Fake Orders empty time.csv', mode='w', newline='', encoding='big5') as f:
    FOrders_writer = csv.writer(f)

    order_id = 1    #訂單編號
    category = 1    #普通方案
    Is_holiday = 0  #是否為假日

    #start_date = "08/02/2017"  #str %d/%m/%Y
    end_date = "18/11/2019"    # 1181
    
    # date = datetime.datetime.strptime(start_date, "%d/%m/%Y")
    end_date = datetime.datetime.strptime(end_date , "%d/%m/%Y")
    # len_of_date = (end_date - date).days + 1

    check_day = []
    start_time_group = 0
    end_time_group = 0
    morning = datetime.datetime.strptime("12:00", format)     # 1: 06:00 ~ 12:00
    noon = datetime.datetime.strptime("14:00", format)        # 2: 12:00 ~ 14:00
    afternoon = datetime.datetime.strptime("18:00", format)   # 3: 14:00 ~ 18:00
    night = datetime.datetime.strptime("23:59", format)       # 4: 18:00 ~ 24:00 , 00:00 ~ 06:00
    midnight = datetime.datetime.strptime("06:00", format)

    ##### Write the Fake Orders #####
    FOrders_writer.writerow(['id', 'locate_id', 'space_id', 'size', 'hours', 'price', 'order_price', 'price_per_size', 'date', 'start', 'end', 'start_time_group', 'end_time_group', 'weekday', 'Is_holiday', 'zipcode', 'category', 'mrt'])
    for i in range(0, len(space_id)):  #space_list

      ##### 計算space的創造時間 ######
      space_creat = datetime.datetime.strptime(creat_time[i], '%d%b%y:%H:%M:%S') #read info like '08FEB17:21:43:04'
      space_creat_date = datetime.datetime.strptime(space_creat.strftime("%d/%m/%Y") , "%d/%m/%Y") #空間建立時間
      len_of_date = (end_date - space_creat_date).days + 1 #int 計算兩個日期間距
      
      ##### 設定房間起始日期 #####
      date = datetime.datetime.strptime(space_creat.strftime("%d/%m/%Y"), "%d/%m/%Y")
      order_start = datetime.datetime.strptime(open_time[i], format) #訂單開始時間
      order_end = datetime.datetime.strptime(close_time[i], format)  #訂單結束時間
      if (order_start >= midnight and order_start < morning):
        start_time_group = 1
      elif (order_start >= morning and order_start < noon):
        start_time_group = 2
      elif (order_start >= noon and order_start < afternoon):
        start_time_group = 3
      else:
        start_time_group = 4

      if (order_end > midnight and order_end <= morning):
        end_time_group = 1
      elif (order_end > morning and order_end <= noon):
        end_time_group = 2
      elif (order_end > noon and order_end <= afternoon):
        end_time_group = 3
      else:
        end_time_group = 4

      ##### 開始產生訂單 #####
      for j in range(0, len_of_date): #根據空間的建立時間，再跑到 31/12/2019
        start = open_time[i]  #重置每日的租用時段
        check_day = list(range(open_day[i], close_day[i]+1))
        
        Is_holiday = 1 if(date.isoweekday() == 6 or date.isoweekday() == 7) else 0

        if (date.isoweekday() in check_day):
          for k in range(start_time_group, end_time_group + 1):
            if (k == 1):
              order_start = datetime.datetime.strptime("09:00", format)
              order_end = morning
            elif (k == 2):
              order_start = morning
              order_end = noon
            elif (k == 3):
              order_start = noon
              order_end = afternoon
            else:
              order_start = afternoon
              order_end = datetime.datetime.strptime("21:00", format)
            ##### 計算訂單租用時間 #####
            diff = order_end - order_start 
            seconds = diff.total_seconds()
            hours = seconds // 3600
            minutes = (seconds % 3600) // 60
            duration = hours + minutes/60  #訂單租用時間
            
            ##### Actual write #####
            FOrders_writer.writerow([order_id, locate_id[i], space_id[i], size[i], duration, price[i], price[i]*duration, price[i]/size[i], date.date().strftime("%d/%m/%Y"), order_start.strftime("%H:%M:%S"), order_end.strftime("%H:%M:%S"), k, k, date.isoweekday(), Is_holiday, zipcode[i], category, mrt[i]])
            order_id += 1
        date = date + datetime.timedelta(days=1)
    f.close()


## Original Order Format

In [0]:
id	space_id	size	hours	price	order_price	price_per_size	start_time_group	End_time_group	weekday	Is_holiday	country_holiday	zipcode	category
9615	1	12	3	300	900	25	0	1	3	0	0	100	1
9617	1	12	4	300	1200	25	4	4	5	0	0	100	1
9618	1	12	4	300	1200	25	4	4	6	1	0	100	1

## Code Test

In [0]:
###  Calculate number of days between two given dates ###
import datetime

date_format = "%H:%M:%S"
order_s = datetime.datetime.strptime('18:00:00', date_format)
order_e = datetime.datetime.strptime('00:00:00', date_format)
delta = (order_e - order_s)
#print(delta.strftime("%H:%M:%S"))
print(delta)
print(abs(delta))
print(datetime.datetime(days=1))
#print(a < b)
#print(order_e)
#print(order_e.strftime("%H:%M:%S"))
#print(type(order_e.strftime("%H:%M:%S")))
# print (delta.days)
# print (type(delta.days))


-1 day, 6:00:00
18:00:00
<class 'datetime.datetime'>


In [0]:
###  Calculate number of days between two given dates ###
import datetime

date_format = "%d/%m/%Y"
a = datetime.datetime.strptime('24/08/2016', date_format)
b = datetime.datetime.strptime('18/11/2019', date_format)
delta = (b - a).days
print(delta)

# print (delta.days)
# print (type(delta.days))


1181


In [0]:

order_start = datetime.datetime.strptime(open_time[i], format)
order_end = datetime.datetime.strptime(close_time[i], format)

diff = order_start - order_end
#diff = datetime.datetime.strptime(str(diff), '%H:%M:%S')

the_time = datetime.time(6,30)
value = the_time.hour + the_time.minute/60.0 + \
            the_time.second/3600.0 + the_time.microsecond/3600000000.0

print(diff)
diff = str(diff)
print(type(diff))


In [0]:
results = ['1', '2', '3']
results = list(map(int, results))
print(results)

[1, 2, 3]


In [0]:
import csv
with open('test.csv', 'w', newline='') as csvfile:
    testwriter = csv.writer(csvfile)
    start = 9 
    #start = '09:00:00'

    print('%02d' % start + ':00:00')
    testwriter.writerow(['start', 'end'])
    for i in range(0, 4):
      testwriter.writerow(['%02d' % start + ':00:00', '%02d' % (start+3) + ':00:00'])
      start += 3

09:00:00


In [0]:
import time
#named_tuple = time.localtime() # get struct_time
time_string = "1 September, 2017"
result = time.strptime(time_string, "%d %B, %Y")

test = result
time_string = time.strftime("%d/%m/%Y", test)
print(time_string)
print(type(test))
print(test[2]+1)



01/09/2017
<class 'time.struct_time'>
2


In [0]:
start = '09:00:00'
aa = 9

print('%02d' % aa)

aa += 3

print('%02d' % aa + ':00:00')


09
12:00:00


In [0]:
space_id = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,31,41,51,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187]

p =[1, 4, 8, 2]
h = 3
o_p = h * p
print(type(o_p))


for i in space_id:
  print(i, type(i))

